In [145]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///data.sqlite')
#connect to the database
con = engine.connect()
# cursor  



  


In [146]:
### Q1: Some of the facilities charge a fee to members, but some do not.
##Please list the names of the facilities that do.###


In [147]:
sql_cmd = """
SELECT name, membercost
FROM Facilities
WHERE (SELECT CAST(membercost AS float)) > 0.0
GROUP BY name
LIMIT 0 , 30 """
rs = con.execute(sql_cmd)
df = pd.DataFrame(rs.fetchall())  
print (rs1.fetchall())
print (df)

[]
                0    1
0  Massage Room 1  9.9
1  Massage Room 2  9.9
2    Squash Court  3.5
3  Tennis Court 1  5.0
4  Tennis Court 2  5.0


In [148]:
###/* Q2: How many facilities do not charge a fee to members? */
sql_cmd1 = """
SELECT COUNT(name)
FROM Facilities
WHERE (SELECT CAST(membercost AS float)) = 0.0"""
rs = con.execute(sql_cmd1)
df = pd.DataFrame(rs.fetchall())  
print (df)


   0
0  4


In [149]:
#/* Q3: How can you produce a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost?
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question. */

sql_cmd2 = """
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE (SELECT CAST(membercost as float)) < (SELECT CAST(monthlymaintenance as float)) * 0.2 """
rs = con.execute(sql_cmd2)
print (rs.fetchall())


[('0', 'Tennis Court 1', '5.0', '200'), ('1', 'Tennis Court 2', '5.0', '200'), ('2', 'Badminton Court', '0.0', '50'), ('3', 'Table Tennis', '0.0', '10'), ('4', 'Massage Room 1', '9.9', '3000'), ('5', 'Massage Room 2', '9.9', '3000'), ('6', 'Squash Court', '3.5', '80'), ('7', 'Snooker Table', '0.0', '15'), ('8', 'Pool Table', '0.0', '15')]


In [150]:
#/* Q4: How can you retrieve the details of facilities with ID 1 and 5?
#Write the query without using the OR operator. */
sql_cmd3 = """
SELECT *
FROM Facilities
WHERE facid IN (1, 5)
"""

#/* Q5: How can you produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100? Return the name and monthly maintenance of the facilities
#in question. */
sql_cmd4 = """
SELECT name, monthlymaintenance,
  CASE WHEN monthlymaintenance > 100 THEN 'expensive'
  ELSE 'cheap' END AS maintenance_cheap_or_expensive
FROM Facilities"""


#/* Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Do not use the LIMIT clause for your solution. */

sql_cmd5= """SELECT firstname, surname, joindate
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members)"""


#/* Q7: How can you produce a list of all members who have used a tennis court?
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name. */
sql_cmd6="""SELECT Facilities.name, CONCAT(Members.firstname, ' ', Members.surname) AS member_name
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE Facilities.name LIKE 'Tennis Court%'
GROUP BY Facilities.name, Members.firstname, Members.surname
ORDER BY Members.firstname"""

#/* Q8: How can you produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30? Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries. */
sql_cmd7="""SELECT Facilities.name,
  CONCAT(Members.firstname, ' ', Members.surname) AS member_name,
  CASE WHEN Members.memid !=0
    THEN Bookings.slots * Facilities.membercost
    ELSE Bookings.slots * Facilities.guestcost
    END AS booking_cost
FROM Bookings
INNER JOIN Members ON Bookings.memid = Members.memid
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE Bookings.starttime LIKE '2012-09-14%'
  AND(
    (Bookings.memid != 0 AND Bookings.slots * Facilities.membercost > 30)
    OR (Bookings.memid = 0 AND Bookings.slots * Facilities.guestcost > 30)
  )
ORDER BY booking_cost DESC"""

#/* Q9: This time, produce the same result as in Q8, but using a subquery. */

sql_cmd8="""SELECT *
FROM (
  SELECT Facilities.name,
  CONCAT(Members.firstname, ' ', Members.surname) AS member_name,
  CASE WHEN Members.memid !=0
    THEN Bookings.slots * Facilities.membercost
    ELSE Bookings.slots * Facilities.guestcost
    END AS booking_cost
  FROM Bookings
  INNER JOIN Members ON Bookings.memid = Members.memid
  INNER JOIN Facilities ON Bookings.facid = Facilities.facid
  WHERE Bookings.starttime LIKE '2012-09-14%'
  ) sub
WHERE sub.booking_cost > 30
ORDER BY sub.booking_cost DESC"""

#/* Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! */

sql_cmd9="""SELECT facility, total_revenue
FROM (
  SELECT Facilities.name AS facility,
  SUM(
    CASE WHEN Bookings.memid !=0
    THEN Bookings.slots * Facilities.membercost
    ELSE Bookings.slots * Facilities.guestcost END) AS total_revenue
  FROM Facilities
  INNER JOIN Bookings ON Bookings.facid = Facilities.facid
  GROUP BY Facilities.name
) AS facility_total_revenue
WHERE total_revenue < 1000
ORDER BY total_revenue"""